In [ ]:
import tensorflow as tf
import numpy as np
import random

1. Create two custom layers in TensorFlow that add two inputs and multiply two inputs
respectively [or any other mathematical operations of your choice(linear or
nonlinear)] Add layer names and other things that are present in other common
layers in TensorFlow. (5pts)


In [ ]:
#AdditionLayer

class AdditionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AdditionLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=-1, keepdims=True)

    def get_config(self):
        config = super(AdditionLayer, self).get_config()
        return config



In [ ]:
#MultiplicationLayer

class MultiplicationLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MultiplicationLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.reduce_prod(inputs, axis=-1, keepdims=True)

    def get_config(self):
        config = super(MultiplicationLayer, self).get_config()
        return config

In [ ]:
import tensorflow as tf

class AddLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AddLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=-1, keepdims=True)

class MultiplyLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MultiplyLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.reduce_prod(inputs, axis=-1, keepdims=True)

# Example usage
input_data = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
add_layer = AddLayer()
multiply_layer = MultiplyLayer()

add_result = add_layer(input_data)
multiply_result = multiply_layer(input_data)

print("Input data:", input_data.numpy())
print("Addition result:", add_result.numpy())
print("Multiplication result:", multiply_result.numpy())


Input data: [[1. 2. 3.]
 [4. 5. 6.]]
Addition result: [[ 6.]
 [15.]]
Multiplication result: [[  6.]
 [120.]]


In [ ]:

class AddLayer(tf.keras.layers.Layer):
    def __init__(self, name=None, **kwargs):
        super(AddLayer, self).__init__(name=name, **kwargs)

    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=-1, keepdims=True)

class MultiplyLayer(tf.keras.layers.Layer):
    def __init__(self, name=None, **kwargs):
        super(MultiplyLayer, self).__init__(name=name, **kwargs)

    def call(self, inputs):
        return tf.reduce_prod(inputs, axis=-1, keepdims=True)

# Example usage
input_data = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
add_layer = AddLayer(name="custom_add_layer")
multiply_layer = MultiplyLayer(name="custom_multiply_layer")

add_result = add_layer(input_data)
multiply_result = multiply_layer(input_data)

print("Input data:", input_data.numpy())
print("Addition result:", add_result.numpy())
print("Multiplication result:", multiply_result.numpy())


Input data: [[1. 2. 3.]
 [4. 5. 6.]]
Addition result: [[ 6.]
 [15.]]
Multiplication result: [[  6.]
 [120.]]


2. Combine these two layers in a third custom layer. Concatenate them or multiply
them etc. (5pts)

In [ ]:
class CombineLayers(tf.keras.layers.Layer):
    def __init__(self, operation="concatenate", **kwargs):
        super(CombineLayers, self).__init__(**kwargs)
        self.operation = operation

    def call(self, inputs):
        if self.operation == "concatenate":
            add_result, multiply_result = inputs
            combined_result = tf.concat([add_result, multiply_result], axis=-1)
        elif self.operation == "multiply":
            add_result, multiply_result = inputs
            combined_result = add_result * multiply_result
        else:
            raise ValueError("Invalid operation")

        return combined_result

# Example usage
input_data = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
add_layer = AddLayer()
multiply_layer = MultiplyLayer()
combine_layer = CombineLayers(operation="concatenate")

add_result = add_layer(input_data)
multiply_result = multiply_layer(input_data)
combined_result = combine_layer([add_result, multiply_result])

print("Input data:", input_data.numpy())
print("Addition result:", add_result.numpy())
print("Multiplication result:", multiply_result.numpy())
print("Combined result (concatenate):", combined_result.numpy())


Input data: [[1. 2. 3.]
 [4. 5. 6.]]
Addition result: [[ 6.]
 [15.]]
Multiplication result: [[  6.]
 [120.]]
Combined result (concatenate): [[  6.   6.]
 [ 15. 120.]]


3. Create a model using these and observe how batch inference works (10 pts)

In [ ]:
input_shape = (None, 3)  # Batch size can vary, and each sample has 3 features
inputs = tf.keras.layers.Input(shape=input_shape)
add_layer = AddLayer(name="custom_add_layer")(inputs)
multiply_layer = MultiplyLayer(name="custom_multiply_layer")(inputs)
combined_layer = CombineLayers(operation="concatenate")([add_layer, multiply_layer])

#simple batch inference
model = tf.keras.models.Model(inputs=inputs, outputs=combined_layer)

# Generate random input data
num_samples = 4
random_input_data = tf.random.normal((num_samples,) + input_shape[1:])

# Perform batch inference
combined_result_batch = model(random_input_data)

print("Random Input Data:")
print(random_input_data.numpy())
print("\nCombined Result (Batch Inference):")
print(combined_result_batch.numpy())

Random Input Data:
[[ 0.603917    0.43872657 -0.70467967]
 [ 0.425284    0.89819497 -0.23498227]
 [ 0.4471985  -1.1060203  -0.3660734 ]
 [-0.35611647 -1.0727587   1.6326913 ]]

Combined Result (Batch Inference):
[[ 0.33796388 -0.18670802]
 [ 1.0884967  -0.08976039]
 [-1.0248952   0.1810638 ]
 [ 0.20381606  0.6237322 ]]


4. Use a predefined layer in Tensorflow to split an input/image into 4x4 = 16 tiles


In [ ]:
# Load an example image (replace with your image loading code)
image_path = '/content/cat.jpg'
image = tf.image.decode_image(tf.io.read_file(image_path))

# Parameters for splitting the image into tiles
num_rows = 4
num_cols = 4
patch_height = image.shape[0] // num_rows
patch_width = image.shape[1] // num_cols

# Reshape the image to have a batch dimension
image_batch = tf.expand_dims(image, axis=0)

# Split the image into tiles using tf.image.extract_patches
patches = tf.image.extract_patches(images=image_batch,
                                   sizes=[1, patch_height, patch_width, 1],
                                   strides=[1, patch_height, patch_width, 1],
                                   rates=[1, 1, 1, 1],
                                   padding='VALID')

# Reshape patches to have the shape (num_patches, patch_height, patch_width, num_channels)

num_patches = num_rows * num_cols
patch_shape = [num_patches, patch_height, patch_width, image.shape[2]]
patches_reshaped = tf.reshape(patches, patch_shape)

# Display the shape of the extracted patches
print("Shape of extracted patches:", patches_reshaped.shape)


Shape of extracted patches: (16, 132, 274, 3)


5. Write a graph data structure in python with random connections between nodes.
(5pts)

In [ ]:
# Create nodes
class GraphNode:
    def __init__(self, name):
        self.name = name
        self.edges = []

    def add_edge(self, edge):
        self.edges.append(edge)

    def __str__(self):
        return self.name

nodes = {
    'A': GraphNode('A'),
    'B': GraphNode('B'),
    'C': GraphNode('C'),
    'D': GraphNode('D'),
    'E': GraphNode('E')
}

# Define connections between nodes
nodes['A'].add_edge(nodes['B'])
nodes['A'].add_edge(nodes['C'])
nodes['A'].add_edge(nodes['D'])
nodes['B'].add_edge(nodes['E'])
nodes['C'].add_edge(nodes['E'])
nodes['D'].add_edge(nodes['E'])
nodes['E'].add_edge(nodes['A'])

# Perform inference in node E using the custom model
input_data = tf.constant([[random.randint(1, 100)]])

add_layer = AddLayer()
multiply_layer = MultiplyLayer()
combine_layer = CombineLayers(operation="concatenate")

add_result = add_layer(input_data)
multiply_result = multiply_layer(input_data)
combined_result = combine_layer([add_result, multiply_result])

# Compare the combined result using TensorFlow's comparison operations
greater_than_200 = tf.greater(combined_result, 200)
less_than_100 = tf.less(combined_result, 100)

if tf.reduce_any(greater_than_200):
    next_node = nodes['B']
elif tf.reduce_any(less_than_100):
    next_node = nodes['C']
else:
    next_node = nodes['D']

print("Input data:", input_data.numpy())
print("Combined result:", combined_result.numpy())
print("Next node based on output:", next_node)
#I've updated the code to use TensorFlow's tf.greater and tf.less for comparison operations and tf.reduce_any to determine if any element in the tensor meets the condition. This should resolve the "ValueError: The truth value of an array..." issue.

Input data: [[30]]
Combined result: [[30 30]]
Next node based on output: C


In [ ]:
class GraphNode:
    def __init__(self, name):
        self.name = name
        self.edges = []

    def add_edge(self, edge):
        self.edges.append(edge)

    def __str__(self):
        return self.name

nodes = {
    'A': GraphNode('A'),
    'B': GraphNode('B'),
    'C': GraphNode('C'),
    'D': GraphNode('D'),
    'E': GraphNode('E')
}

# Define connections between nodes
nodes['A'].add_edge(nodes['B'])
nodes['A'].add_edge(nodes['C'])
nodes['A'].add_edge(nodes['D'])
nodes['B'].add_edge(nodes['E'])
nodes['C'].add_edge(nodes['E'])
nodes['D'].add_edge(nodes['E'])
nodes['E'].add_edge(nodes['A'])


In [ ]:
# Print connections between nodes
for node, graph_node in nodes.items():
    edge_names = ', '.join([edge.name for edge in graph_node.edges])
    print(f"{node}: {edge_names}")

A: B, C, D
B: E
C: E
D: E
E: A


6. Two-part question
a. Do inference in one of the nodes and write any rules (>,<,==) to decide the
next node based on the output (5 pts)

In [ ]:
output_value = random.randint(1, 10)
input_node = nodes['A']

# Decide the next node based on the output value using rules
if output_value > 5:
    next_node = input_node.edges[0]
elif output_value < 5:
    next_node = input_node.edges[1]
else:
    next_node = input_node.edges[2]

print("Output value:", output_value)
print("Input node:", input_node)
print("Next node based on output:", next_node)

Output value: 5
Input node: A
Next node based on output: D


You can choose to use the custom model you have created in questions 1 or
2 inside the node. The input can be a random input created using
random.randint(1,100). You can use any error correction techniques
we used before. (5pts)


In [ ]:
# Custom Layers
class AddLayer(tf.keras.layers.Layer):
    def __init__(self, name=None, **kwargs):
        super(AddLayer, self).__init__(name=name, **kwargs)

    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=-1, keepdims=True)

class MultiplyLayer(tf.keras.layers.Layer):
    def __init__(self, name=None, **kwargs):
        super(MultiplyLayer, self).__init__(name=name, **kwargs)

    def call(self, inputs):
        return tf.reduce_prod(inputs, axis=-1, keepdims=True)

# Define a custom model using the custom layers
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.add_layer = AddLayer()
        self.multiply_layer = MultiplyLayer()

    def call(self, inputs):
        add_result = self.add_layer(inputs)
        multiply_result = self.multiply_layer(inputs)
        combined_result = tf.concat([add_result, multiply_result], axis=-1)
        return combined_result

# Create a node with the custom model
class GraphNode:
    def __init__(self, name, model):
        self.name = name
        self.model = model

    def __str__(self):
        return self.name

# Create nodes with custom models
nodes = {
    'A': GraphNode('A', CustomModel()),
    'B': GraphNode('B', None),
    'C': GraphNode('C', None),
    'D': GraphNode('D', None),
    'E': GraphNode('E', None)
}

# Define connections between nodes
nodes['A'].model.build(input_shape=(None, 2))  # Build the model
nodes['A'].model.summary()  # Display model summary

# Perform inference in node A with a random input
input_data = tf.constant([[random.randint(1, 100), random.randint(1, 100)]], dtype=tf.float32)
output_data = nodes['A'].model(input_data)

print("Input data:", input_data.numpy())
print("Output data:", output_data.numpy())


Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 add_layer_3 (AddLayer)      multiple                  0         
                                                                 
 multiply_layer_3 (MultiplyL  multiple                 0         
 ayer)                                                           
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
Input data: [[59.  9.]]
Output data: [[ 68. 531.]]
